In [1]:
import gc
import json
import math
import os
import pickle
import random
import re
import sys

import numpy as np
from charset_normalizer import from_path
from gensim.corpora import Dictionary
import nltk
from tqdm import tqdm

"""
Se importan las librerias que se necesiten, 
si se quiere ejecutar el notebook, se recomienda crear la carpeta de data, y poner ahi los files como se describe

"""
ACTUAL_PATH = os.getcwd()
# Donde esta el 20 News
PATH_20N = os.path.join(ACTUAL_PATH, "data/20news-18828")
# Donde se encuentra el BAC
PATH_BAC = os.path.join(ACTUAL_PATH, "data/BAC/blogs")
# Donde se van a guardar los files que se van obteniendo
PATH_FINAL_FILES = os.path.join(ACTUAL_PATH, "data/final_files")
# Numero de grupo (realmente como no hay pues simplemente se pusimos nuestros nombres)
GRUPO = "Erich_Carlos"

## Clases para los n gramas 

In [2]:
class UnigramModel:
    """ 
    Modelo de unigramas, 
    """
    def __init__(self, filename: str, file_is_training = True):
        """Makes the diccionary that the model needs to work,
        ge
        Args:
            filename (str): Nombre del archivo a procesar
            file_is_training (bool, optional): 
                Indica cómo manejar el archivo de entrada.  
                - Si es False, se carga el objeto ya procesado desde un archivo `.pickle`.  
                - Si es True, el archivo se procesa desde cero.  

        """
        print(filename)
        file = self.get_pickle(filename)
        if file_is_training:
            self.word_counter_20N = {}
            for sentence in file:
                for word in sentence:
                    self.word_counter_20N[word] = self.word_counter_20N.get(word, 0) + 1
            self.total_words = sum(self.word_counter_20N.values())
            self.V = len(self.word_counter_20N)
        else:
            self.word_counter_20N = file["word_counter_20N"]
            self.total_words = file["total_words"]
            self.V = file["V"]

        self.total_words = sum(self.word_counter_20N.values())
        self.V = len(self.word_counter_20N)
    
    def get_pickle(self, filename: str):
        """
        Abre un file en formato .pickle, 
        dentro de PATH_FINAL_FILES y lo devuelve.

        Args:
            filename (str): Nombre del file a abrir

        """
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "rb") as f:
            sentences = pickle.load(f)
        return sentences
    def generate_unigrams(self, filename: str):
        """Genera los unigramas en un archivo (jsonl) es lo que se 
        espera

        Args:
            filename (str): Nombre del archivo
        """
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "w", encoding="utf-8") as f:
            for word in self.word_counter_20N.keys():
                prob = self.get_prob(word)
                f.write(json.dumps({"word": word, "probability": prob}) + "\n")
    def get_prob(self, word: str) -> float:
        """
        Calcula la probabilidad de un unigrama.  
        Si la palabra existe en el vocabulario V, se devuelve su probabilidad.  
        En caso contrario, se asigna al token <UNK>.  

        Args:
            word (str): Palabra a consultar.  

        Returns:
            float: Probabilidad asociada a la palabra.  
        """
        if word.lower() in self.word_counter_20N.keys():
            prob = self.word_counter_20N[word] / self.total_words
        else:
            prob = self.word_counter_20N["<UNK>"] / self.total_words
        return prob
    def get_next_token(self) -> str:
        """ Genera un token según las probabilidades unigramales."""
        probabilities = [self.get_prob([self.token_of(k)]) for k in range(self.V)]
        probs = [math.exp(p) for p in probabilities]
        index = random.choices(range(self.V), weights=probs, k=1)[0]
        return self.token_of(index)
    
    def generate_sentences(self, limit:int= 50) -> list[str]:
        """ Genera una sentencia basado en unigramas

        Args:
            limit (int, optional): limite de palabras a predecir. Defaults to 50.

        Returns:
            list[str]: sentencia en una lista de strings.
        """
        sentence = ["<s>"]
        for _ in range(limit):
            token = self.get_next_token()
            if token == "</s>":
                break
            sentence.append(token)    
    def generate_bigrams(self, filename: str):
        """
        Genera y guarda las probabilidades de todos los bigramas observados y no observados,
        escribiéndolos progresivamente en un archivo JSONL legible.

        Para cada par (i, j) en la matriz `self.V`, se obtiene el token correspondiente 
        mediante `self.token_of`, se calcula su probabilidad con `self.get_prob([w1, w2])` 
        y se escribe directamente en el archivo.

        Args:
            filename (str): Nombre del archivo de salida (.jsonl) donde se guardarán 
                            las probabilidades de los bigramas.

        Efectos:
            Crea un archivo JSONL en PATH_FINAL_FILES/filename donde cada línea 
            tiene la forma:
            {"w1": "...", "w2": "...", "probabilidad": ...}
        """

        filepath = os.path.join(PATH_FINAL_FILES, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            iterator = tqdm(
                [(i, j) for i in range(self.V) for j in range(self.V)],
                desc="Generando bigramas",
                unit="bigrama",
                total=self.V * self.V,
                leave=True,
                file=sys.stdout
            )

            for i, j in iterator:
                w1 = self.token_of(i)
                w2 = self.token_of(j)
                prob = self.get_prob([w1, w2])

                record = {"w1": w1, "w2": w2, "probabilidad": prob}
                f.write(json.dumps(record, ensure_ascii=False) + "\n")
        sentence.append("</s>")
        return " ".join(sentence)
    def save_model(self, filename: str):
        """
        Guarda el modelo de unigramas entrenado en un archivo `.pickle`.

        El archivo contendrá:
        - word_counter_20N: Diccionario de conteos de palabras.
        - total_words: Número total de palabras en el corpus.
        - V: Tamaño del vocabulario.

        Args:
            filename (str): Nombre del archivo de salida.
        """
        payload = {
            "word_counter_20N": self.word_counter_20N,
            "total_words": self.total_words,
            "V": self.V
        }
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "wb") as f:
            pickle.dump(payload, f, protocol=pickle.HIGHEST_PROTOCOL)
    
class BigramModel:
    """Bigram Model
    """
    def __init__(self, filename:str, file_is_training=True):
        """
        Inicializador del modelo de bigramas.

        En este paso se construyen varias estructuras necesarias:

        - dictionary: objeto que permite mapear palabras ↔ tokens.
        - V: número total de tokens en el corpus (palabras + caracteres especiales).
        - matrix: diccionario que guarda el conteo de ocurrencias de los bigramas 
            observados. (La matriz completa sería inviable de almacenar).
        - row_sums: para agilizar el cálculo de probabilidades se guarda, para cada 
            token, la suma total de sus ocurrencias como primer elemento en un bigrama. 
            De esta forma, el denominador de la probabilidad condicional ya está 
            precomputado y no es necesario recalcularlo en cada consulta.
        """

        data = self._load_pickle(filename)

        if file_is_training:
            self.dictionary = Dictionary(data)
            self.V = len(self.dictionary)

            self.matrix = {}                  
            self.row_sums = {}            

            for sentence in data:
                for i in range(len(sentence)):
                    w_idx = self._word_index(sentence[i])
                    if i < len(sentence) - 1:
                        w_next_idx = self._word_index(sentence[i + 1])
                        key = (w_idx, w_next_idx)
                        self.matrix[key] = self.matrix.get(key, 0) + 1
                        self.row_sums[w_idx] = self.row_sums.get(w_idx, 0) + 1

        else:
            self.dictionary = data["dictionary"]
            self.V = data["V"]
            self.matrix = dict(data["matrix"])
            self.row_sums = dict(data["row_sums"])

    def _word_index(self, word: str) -> int:
        """
        Devuelve el ID asociado a un token. 
        Si el token no existe en el diccionario, se asigna el ID correspondiente de <UNK>.

        Args:
            word (str): Palabra o token cuyo ID se desea obtener.

        Returns:
            int: ID de la palabra o, en caso de no estar en el diccionario, 
                el ID de <UNK>.
        """
        tid = self.dictionary.token2id.get(word)
        if tid is None:
            tid = self.dictionary.token2id["<UNK>"]
        return tid

    def _load_pickle(self, filename: str):
        """Carga de un pickle

        Args:
            filename (str): Nombre del file

        Returns:
            _type_: Estructura que posea el pickle
        """
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "rb") as f:
            return pickle.load(f)

    def save_model(self, filename: str):
        """Guarda el modelo, para no tener que
        volver a recalcular.

        Args:
            filename (str): Nombre del file en el cual se va a guardar el modelo
        """
        payload = {
            "dictionary": self.dictionary,
            "V": self.V,
            "matrix": dict(self.matrix),
            "row_sums": dict(self.row_sums)
        }
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "wb") as f:
            pickle.dump(payload, f, protocol=pickle.HIGHEST_PROTOCOL)

    def token_of(self, idx: int) -> str:
        """Obtiene el token asociado a una ID

        Args:
            idx (int): ID del token

        Returns:
            str: Token asociado a la ID
        """
        return self.dictionary.id2token.get(idx, "<UNK>")

    def get_prob(self, words: list[str]) -> float:
        """Se obtiene la probabilidad de una lista palabras
        [w1,w2]

        Args:
            words (list[str]): Lista de palabras sobre la que 
            se obtiene 

        Returns:
            float: probabilidad de w1, w2
        """
        m_i = self._word_index(words[0])
        m_j = self._word_index(words[1])
        c_bigram = self.matrix.get((m_i, m_j), 0)
        row_sum  = self.row_sums.get(m_i, 0)
        return np.log((c_bigram + 1) / (row_sum + self.V))

    def generate_bigrams(self, filename: str):
        """
        Genera y guarda SOLO las probabilidades de los bigramas OBSERVADOS
        (claves en self.matrix), incluyendo aquellos que involucren <UNK>.

        Args:
            filename (str): Nombre del archivo de salida (.jsonl).
        """
        filepath = os.path.join(PATH_FINAL_FILES, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            for (i, j), _count in self.matrix.items():
                w1 = self.token_of(i)
                w2 = self.token_of(j)
                prob = self.get_prob([w1, w2])
                record = {"w1": w1, "w2": w2, "probabilidad": prob}
                f.write(json.dumps(record, ensure_ascii=False) + "\n")
    def get_next_token(self, words:list[str]) -> str:
        """
        Predice el siguiente token a partir del modelo de bigramas.

        A partir del token actual (`words[0]`), se calculan las probabilidades
        de transición hacia todos los tokens del vocabulario. Luego, se elige 
        aleatoriamente un token ponderado por dichas probabilidades.

        Args:
            words (list[str]): Lista que contiene el token actual en la primera posición.

        Returns:
            str: El siguiente token predicho.
        """
        probabilities = []
        for k in range(self.V):
            probabilities.append(self.get_prob([words[0], self.token_of(k)]))
        probs = [math.exp(p) for p in probabilities] 
        index = random.choices(range(self.V), weights=probs, k=1)[0]
        return self.token_of(index)
    
    def generate_sentences(self, words:list[str], limit= 50) -> list[str]:
        """
        Genera una oración utilizando un modelo de bigramas.

        La generación inicia a partir del token en `words[0]`. En cada paso, 
        se predice el siguiente token con `get_next_token`, se agrega a la 
        oración y se actualiza el contexto. El proceso termina al alcanzar el 
        token de fin de secuencia `</s>` o al superar el límite de tokens.

        Args:
            words (list[str]): Lista inicial con el token de partida en la 
                primera posición.
            limit (int, optional): Número máximo de tokens a generar. 
                Por defecto 50.

        Returns:
            str: Oración generada.
        """
        i = 0
        sentence = words[0]
        predicted_token = self.get_next_token(words)
        words[0] = predicted_token
        while i != limit or predicted_token == "</s>":
            predicted_token = self.get_next_token(words)
            sentence += " " + predicted_token
            words[0] = predicted_token
            i+=1
        return sentence
    
class TrigramModel:
    """
    Inicializa el modelo de trigramas.

    Si `file_is_training` es True, procesa los datos para construir el diccionario 
    y las estructuras necesarias para calcular probabilidades.  
    Si es False, carga el modelo previamente entrenado desde un archivo `.pickle`.

    Args:
        filename (str): Nombre del archivo de entrada.
        file_is_training (bool, optional): 
            - True: procesa los datos desde cero.  
            - False: carga un modelo ya procesado.  
    """
    def __init__(self, filename, file_is_training=True):
        data = self._load_pickle(filename)
        if file_is_training:
            self.dictionary = Dictionary(data)
            self.dictionary.add_documents([["<UNK>"]])
            self.V = len(self.dictionary)
            self.matrix_trigram = {}      
            self.pair_sums = {}           
            for sent in data:
                ids = [self._word_index(w) for w in sent]
                for t in range(len(ids) - 2):
                    i, j, k = ids[t], ids[t+1], ids[t+2]
                    key3 = (i, j, k)
                    key2 = (i, j)
                    self.matrix_trigram[key3] = self.matrix_trigram.get(key3, 0) + 1
                    self.pair_sums[key2] = self.pair_sums.get(key2, 0) + 1
        else:
            self.dictionary     = data["dictionary"]
            self.V              = data["V"]
            self.matrix_trigram = dict(data["matrix_trigram"])
            self.pair_sums      = dict(data["pair_sums"])

    def _word_index(self, word: str) -> int:
        """
        Devuelve el ID asociado a una palabra.

        Si la palabra no existe en el diccionario, devuelve el ID de <UNK>.

        Args:
            word (str): Palabra a consultar.

        Returns:
            int: ID asociado a la palabra o al token <UNK>.
        """
        tid = self.dictionary.token2id.get(word)
        if tid is None:
            tid = self.dictionary.token2id["<UNK>"]
        return tid

    def _load_pickle(self, filename: str):
        """
        Carga un archivo `.pickle` desde PATH_FINAL_FILES.

        Args:
            filename (str): Nombre del archivo a cargar.

        Returns:
            object: Contenido del pickle (corpus o modelo guardado).
        """
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "rb") as f:
            return pickle.load(f)

    def save_model(self, filename: str):
        """
        Guarda el modelo entrenado en un archivo `.pickle`.

        El archivo incluye:
        - Diccionario de tokens.
        - Tamaño del vocabulario.
        - Conteo de trigramas observados.
        - Conteo de pares de tokens.

        Args:
            filename (str): Nombre del archivo de salida.
        """
        payload = {
            "dictionary": self.dictionary,
            "V": self.V,
            "matrix_trigram": dict(self.matrix_trigram),
            "pair_sums": dict(self.pair_sums),
        }
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "wb") as f:
            pickle.dump(payload, f, protocol=pickle.HIGHEST_PROTOCOL)

    def token_of(self, idx: int) -> str:
        """
        Devuelve el token asociado a un ID.

        Args:
            idx (int): ID del token.

        Returns:
            str: Token correspondiente o <UNK> si no existe.
        """
        return self.dictionary.id2token.get(idx, "<UNK>")

    def get_prob(self, words: list[str]) -> float:
        """
        Calcula la probabilidad de un trigrama.

        Usa la fórmula:
            P(w_k | w_i, w_j) = (conteo(i, j, k)) / (conteo(i, j) + V)

        con suavizado de Laplace.

        Args:
            words (list[str]): Lista con tres tokens [w_i, w_j, w_k].

        Returns:
            float: Probabilidad logarítmica del trigrama.
        """
        i = self._word_index(words[0])
        j = self._word_index(words[1])
        k = self._word_index(words[2])
        V = self.V
        c_ijk = self.matrix_trigram.get((i, j, k), 0)
        denom = self.pair_sums.get((i, j), 0)
        return float(np.log((c_ijk + 1) / (denom + V)))
    def generate_trigrams(self, filename: str):
        """
        Guarda SOLO trigramas OBSERVADOS (claves de self.matrix_trigram),
        incluyendo aquellos que involucren <UNK>.

        Args:
            filename (str): Nombre del archivo de salida (.jsonl).
        """
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "w", encoding="utf-8") as f:
            for (i, j, k), _count in self.matrix_trigram.items():
                w1, w2, w3 = self.token_of(i), self.token_of(j), self.token_of(k)
                prob = self.get_prob([w1, w2, w3])
                record = {"w1": w1, "w2": w2, "w3": w3, "probabilidad": prob}
                f.write(json.dumps(record, ensure_ascii=False) + "\n")
    
    def get_next_token(self, words:list[str]) -> str:
        """
        Predice el siguiente token a partir del modelo de trigramas.

        A partir del par actual `(words[0], words[1])`, calcula las probabilidades
        de transición hacia todos los tokens del vocabulario y selecciona 
        aleatoriamente el siguiente token según dichas probabilidades.

        Args:
            words (list[str]): Lista de dos tokens que sirven como contexto.

        Returns:
            str: El siguiente token predicho.
        """
        probabilities = []
        if (words[0], words[1]) in self.pair_sums:
            for k in range(self.V):
                probabilities.append(self.get_prob([words[0], words[1], self.token_of(k)]))
        probs = [math.exp(p) for p in probabilities] 
        return random.choices(range(self.V), weights=probs, k=1)[0]
    def generate_sentences(self, words:list[str], limit= 50) -> list[str]:
        """
        Genera una oración utilizando un modelo de trigramas.

        La generación comienza con dos palabras iniciales (`words[0]` y `words[1]`).  
        En cada paso se predice el siguiente token con `get_next_token`, se añade 
        a la oración y se actualiza el contexto.  
        El proceso se detiene al alcanzar el token de fin de secuencia `</s>` 
        o al llegar al número máximo de tokens (`limit`).

        Args:
            words (list[str]): Lista inicial con dos tokens de contexto.
            limit (int, optional): Número máximo de tokens generados. 
                Por defecto 50.

        Returns:
            str: Oración generada.
        """
        i = 0
        sentence = " ".join(words)
        predicted_token = self.get_next_token(words)
        words[0] = words[1]
        words[1] = predicted_token
        while i != limit or predicted_token == "<s>":
            predicted_token = self.get_next_token(words)
            sentence += " " + predicted_token
            words[0] = words[1]
            words[1] = predicted_token
        return sentence

## Funciones para calcular perplejidades

In [3]:
import math
def perplexity_unigram(model: UnigramModel, filename:str) -> float:
    """
    Calcula la perplejidad de un modelo de unigramas sobre un corpus de prueba.  

    La perplejidad mide qué tan bien el modelo predice un conjunto de oraciones.  
    Se calcula como:  

        PP = exp( - (1/T) * Σ log P(w_i) )  

    donde T es el número total de palabras en el corpus.  

    Args:
        model (UnigramModel): Modelo de unigramas sobre el que se evalúa.  
        filename (str): Nombre del archivo de prueba (.pickle) que contiene las oraciones.  

    Returns:
        float: Valor de la perplejidad.  
                Devuelve `inf` si alguna palabra tiene probabilidad 0 o si el corpus está vacío.  
    """
    sentences = model.get_pickle(filename)
    log_sum = 0.0
    T = 0
    for s in sentences:
        for w in s:
            p = model.get_prob(w)
            if p == 0.0:
                return float("inf")
            log_sum += math.log(p)
            T += 1
    return math.exp(-log_sum / T) if T else float("inf")

def perplexity_bigram_from_model(model:BigramModel, filename:str):
    """
    Calcula la perplejidad de un modelo de bigramas sobre un corpus de prueba.  

    La perplejidad mide la capacidad del modelo para predecir secuencias de palabras:  

        PP = exp( - (1/T) * Σ log P(w_i | w_{i-1}) )  

    Args:
        model (BigramModel): Modelo de bigramas sobre el que se evalúa.  
        filename (str): Nombre del archivo de prueba (.pickle) que contiene las sentencias.  

    Returns:
        float: Valor de la perplejidad.  
                Devuelve `inf` si el corpus está vacío.  
    """
    sentences = model._load_pickle(filename)
    log_sum = 0.0
    T = 0
    for s in sentences:
        for prev, cur in zip(s[:-1], s[1:]):
            log_p = model.get_prob([prev, cur])  
            log_sum += log_p
            T += 1
    return math.exp(-log_sum / T) if T else float("inf")

def perplexity_trigram_from_model(model:TrigramModel, filename:str):
    """
    Calcula la perplejidad de un modelo de trigramas sobre un corpus de prueba.  

    La perplejidad se calcula como:  

        PP = exp( - (1/T) * Σ log P(w_i | w_{i-2}, w_{i-1}) )  

    Args:
        model (TrigramModel): Modelo de trigramas sobre el que se evalúa.  
        filename (str): Nombre del archivo de prueba (.pickle) que contiene las oraciones.  

    Returns:
        float: Valor de la perplejidad.  
                Devuelve `inf` si el corpus está vacío.  
    """
    sentences = model._load_pickle(filename)
    log_sum = 0.0
    T = 0
    for s in sentences:
        for prev_2, prev_1 ,cur in zip(s[:-2], s[1:-1], s[2:]):
            log_p = model.get_prob([prev_2, prev_1, cur])  
            log_sum += log_p
            T += 1
    return math.exp(-log_sum / T) if T else float("inf")

## Carga de los modelos

In [4]:
modelo_de_unigramas_20N = UnigramModel(f"20N_{GRUPO}_unigram_model.pkl", file_is_training=False)
modelo_de_unigramas_BAC = UnigramModel(f"BAC_{GRUPO}_unigram_model.pkl", file_is_training=False)

modelo_de_bigramas_20N = BigramModel(f"20N_{GRUPO}_bigram_model.pkl", file_is_training=False)
modelo_de_bigramas_BAC = BigramModel(f"BAC_{GRUPO}_bigram_model.pkl", file_is_training=False)

modelo_de_trigramas_20N = TrigramModel(f"20N_{GRUPO}_trigram_model.pkl", file_is_training=False)
modelo_de_trigramas_BAC = TrigramModel(f"BAC_{GRUPO}_trigram_model.pkl", file_is_training=False)

20N_Erich_Carlos_unigram_model.pkl
BAC_Erich_Carlos_unigram_model.pkl


In [ ]:
from tabulate import tabulate 

stats = [
    ["BAC - Unigram", perplexity_unigram(modelo_de_unigramas_BAC, f"BAC_{GRUPO}_testing.pkl")],
    ["20N - Unigram", perplexity_unigram(modelo_de_unigramas_20N, f"20N_{GRUPO}_testing.pkl")],
    ["20N - Bigram", perplexity_bigram_from_model(modelo_de_bigramas_20N, f"20N_{GRUPO}_testing.pkl")],
    ["BAC - Bigram", perplexity_bigram_from_model(modelo_de_bigramas_BAC, f"BAC_{GRUPO}_testing.pkl")],
    ["20N - Trigram", perplexity_trigram_from_model(modelo_de_trigramas_20N, f"20N_{GRUPO}_testing.pkl")],
    ["BAC - Trigram", perplexity_trigram_from_model(modelo_de_trigramas_BAC, f"BAC_{GRUPO}_testing.pkl")],
]

print(tabulate(stats, headers=["Modelo", "Perplejidad"], tablefmt="fancy_grid", floatfmt=".4f"))